In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from keras.utils import np_utils

In [ ]:
nn_frame = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/group_project/datasets/model_frame.csv')

In [ ]:
nn_frame = nn_frame.drop(columns=['tracking_id'])

In [ ]:
X = nn_frame.drop(columns=['outcome_type','outcome_subtype'])
y = nn_frame['outcome_type']

# making our label categorical
encoder = LabelEncoder()
encoded_y = encoder.fit_transform(y)
y = np_utils.to_categorical(encoded_y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, stratify=y)
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [ ]:
def baseline_nn():
  model = Sequential()
  #model.add(BatchNormalization(input_dim=X_train.shape[1],))

  model.add(Dense(1024	, activation='relu',input_dim=X_train.shape[1],kernel_regularizer=l2(.001)))
  model.add(Dropout(0.25))

  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(2048	, activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(9, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
model = baseline_nn()
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, 
                           verbose=1, mode='auto')
history = model.fit(X_train_sc, y_train, validation_data=(X_test_sc, y_test),epochs=40, batch_size=16, callbacks=[early_stop])

Epoch 1/40
5996/5996 [==============================] - 158s 26ms/step - loss: 1.2441 - accuracy: 0.6718 - val_loss: 1.1916 - val_accuracy: 0.6882
Epoch 2/40
5996/5996 [==============================] - 158s 26ms/step - loss: 1.1782 - accuracy: 0.6941 - val_loss: 1.1533 - val_accuracy: 0.7046
Epoch 3/40
5996/5996 [==============================] - 157s 26ms/step - loss: 1.1427 - accuracy: 0.6986 - val_loss: 1.0806 - val_accuracy: 0.7080
Epoch 4/40
5996/5996 [==============================] - 158s 26ms/step - loss: 1.0958 - accuracy: 0.7021 - val_loss: 1.0623 - val_accuracy: 0.7064
Epoch 5/40
5996/5996 [==============================] - 158s 26ms/step - loss: 1.0689 - accuracy: 0.7034 - val_loss: 1.0200 - val_accuracy: 0.7090
Epoch 6/40
5996/5996 [==============================] - 158s 26ms/step - loss: 1.0430 - accuracy: 0.7057 - val_loss: 0.9993 - val_accuracy: 0.7145
Epoch 7/40
5996/5996 [==============================] - 162s 27ms/step - loss: 1.0289 - accuracy: 0.7069 - val_loss: 1

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
plt.figure(figsize=(12, 8))
plt.plot(train_loss, label='Training loss', color='violet')
plt.plot(test_loss, label='Testing loss', color='lavender')
plt.legend();